In [3]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [4]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

## Assignment 4

## Task 1 

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [5]:
np.set_printoptions(precision=4,suppress=True) # easier to check values rn when we dont care too much about the exactness

In [6]:
# in minimization think probability of being wrong, in max probability of acceptance
# probability of sample A dominating sample B in max is P(A > B) is
def prob_selection(a, b, sigma_a, sigma_b):
    #sigma_b = sigma_b + 0.01 # just to prevent division by zero
    # seems not to be the correct way of hanlding sigma 0
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    top = -2.5*m / np.sqrt(2 + 2*s**2)
    bot = 1 + np.exp(top)
    return 1 / bot

# b clearly dominates
a = np.array([1.0, 1.0])
b = np.array([0.5, 1.5])
sigma_a = 0.001
sigma_b = 0.001

prob_selection(a, b, sigma_a, sigma_b)

array([1., 0.])

In [7]:
def prob_sel2(a, b, sigma_a, sigma_b):
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    #m = m * 10
    print(m)
    
    bot = 0.8*np.sqrt(2 + 2*s**2)
    return 0.5*(1 + np.tanh(m/bot))
    
    
    
# a1 clearly dominates b1, b1 clearly dominates a2
a = np.array([1.0])
b = np.array([0.5])
sigma_a = 1.
sigma_b = 1.

prob_sel2(a, b, sigma_a, sigma_b)

[0.5]


array([0.6514])

In [8]:
# a1 clearly dominates b1, b1 clearly dominates a2
a = np.array([1.0])
b = np.array([0.5])
sigma_a = 0.1
sigma_b = 0.1

prob_sel2(a, b, sigma_a, sigma_b)

[5.]


array([0.9981])

So we can see that probability of being wrong with first items (1 and 2) by choosing a is 0.22. Again for second items (1,5) thats 0.007.  And also with 3rd items (3,1) its 0.92. So we can clearly see its working properly.

## Task 2

Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)

- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [9]:
dtlz2_3 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

In [10]:
def create_samples(dec_dim, samples):
    # create samples 
    x = lhs(dec_dim, samples)     
    return np.asarray(x)

In [11]:
x = create_samples(5, 50)
print(x.shape)
eval_results = dtlz2_3.evaluate(x)
y = eval_results.objectives
print(y[:5])

(50, 5)
[[1.0668 0.5408 0.7942]
 [0.1699 0.036  1.1112]
 [0.8094 0.78   1.097 ]
 [0.6699 0.0956 0.9318]
 [0.2268 0.7223 0.7839]]


In [12]:
print(y[:,0])

[1.0668 0.1699 0.8094 0.6699 0.2268 0.0043 0.2729 0.8219 0.7401 1.0812
 0.559  0.5981 0.0238 0.1004 0.5951 0.4605 0.3882 0.1644 0.0413 0.1049
 0.2666 0.7666 0.9582 0.1136 0.317  0.3608 0.5692 0.3433 0.3233 0.756
 0.6436 0.23   0.2251 0.0075 0.4687 0.903  0.5115 0.683  0.6598 0.9424
 0.738  0.6803 1.1731 0.282  0.0087 1.0503 0.6059 0.7074 0.017  1.0741]


In [13]:
## build surrogates

kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 20.0), nu=1.5)
    
y1 = y[:,0]
gpr1 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y1)

y2 = y[:,1]
gpr2 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y2)

y3 = y[:,2]
gpr3 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y3)

In [16]:
## resampling

x_samples = create_samples(5, 20)

# predict
y1_sample, y1_std = gpr1.predict(x_samples, return_std=True) 
print(np.mean(y1_std))

# predict
y2_sample, y2_std = gpr2.predict(x_samples, return_std=True) 
print(np.mean(y2_std))
# predict
y3_sample, y3_std = gpr3.predict(x_samples, return_std=True) 
print(np.mean(y3_std))

y_samples = np.column_stack((y1_sample, y2_sample, y3_sample))
#y_samples.reshape(1,-1)
y_samples.shape

0.06867311849039004
0.07332612206176277
0.058133660004295616


(20, 3)

In [17]:
print(y_samples)

[[ 0.0163  0.455   0.9597]
 [ 1.1093  0.3884  0.3568]
 [ 0.2481  0.2141  1.3462]
 [ 0.7222  0.137   0.9658]
 [ 0.9132  0.5184  0.6437]
 [ 0.6316  0.9443  0.5498]
 [ 0.2316  0.5148  1.2019]
 [ 0.9392  0.4815  0.565 ]
 [ 0.0943  0.245   1.3302]
 [ 0.8478  0.2146  0.8207]
 [ 0.1858  0.8328  0.7581]
 [ 0.3354  1.2419  0.2287]
 [ 0.816   0.5767  0.7197]
 [ 0.1365  0.0025  1.3502]
 [ 0.1872  0.1615  1.0186]
 [ 0.1424  1.1369  0.2426]
 [ 0.3743  0.391   1.0853]
 [ 0.4682  0.9476 -0.009 ]
 [ 0.5401  0.6512  0.9633]
 [ 0.6609  0.0997  0.9897]]


In [19]:
def sum_of_rank_pos(n):
    return n*(n - 1)/2

sum_of_rank_pos(y_samples.shape[0])

190.0

In [20]:
A = [0.2, 0.9]  # tässä on a1 a2. kun lasketaan prod P (Ai < bi) i to n, tulee P(A < B)
B = [0.3, 0.5]
C = [0.5, 0.3]
D = [0.8, 0.2]
E = [0.85, 0.5]
F = [0.9, 0.95]

sols = np.array([A, B, C, D, E, F])
print(sols.shape)
ff = np.zeros(sols.shape[0])
# this being eq21
#for i in range(ff.shape[0]):
    #for j in range(ff.shape[0]):
        #ff[i] += smaller(sols[j], sols[i])
    #ff[i] -= 0.5

(6, 2)


## DONE:

Single objective ranking works with uncertainty 0 and uncertainty 1. So prob_sel2 works
MO ranking with uncertainty 0 works

mo ranking with uncertainty works

## TODO:

rest of the assigment using some data

ask/find out about the bug with m.

In [73]:

## Need to:
## either multiply m by 10, or multiply sigmas by 10, to get similar results than in example
##


def prob_sel2(a, b, sigma_a, sigma_b):
    s = sigma_a / sigma_b
    #print("S",s)
    #s = 1.
    m = (a - b) / sigma_b
    #print("M",m)
    m = m *10

    bot = 0.8*np.sqrt(2 + 2*s**2)
    res = 0.5*(1 + np.tanh(m/bot))
    res = np.prod(res)
    return res


# note 18.27 approx with uncertainty of 0 works
def sof_prob_dom_multi2(fit, j, sigma_a, sigma_b):
    summa = 0
    # should just go over the multi
    for i in range(fit.shape[0]):
        for ii in range(fit.shape[1]):
            
            fits = fit[i][ii]
            if j[ii] == fits:
                #summa += 0.25
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("eq", s)
                summa += 0.5*s
            elif j[ii] > fits:
                #summa += 0.5
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("doms",s)
                summa += 0.5*s
            elif j[ii] < fits:
                #summa += 0
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("does not dom",s)
                summa += 0.5*s
            else:
                print("should not get here")
        
    summa -= 0.5
    # this seems to need to be -5.5 for 5 var, 3 obj
    
    return summa

###  this
If the tness values are non-dominated, the corresponding
probabilities are P (A > B ) = 0, P (A < B ) = 0, and P (A  B ) = 1. We have
now lost the value 1 from the probability of domination calculations, therefore

In [74]:
s2 = []
sigma_a, sigma_b = 0.001, 0.001

for i in range(len(sols)):
    s2.append(sof_prob_dom_multi2(sols, sols[i], sigma_a, sigma_b))
s2

[2.0, 1.75, 1.5, 1.5, 3.25, 5.0]

In [75]:
np.sum(s2)

15.0

In [76]:
s3 = []
sigma_a, sigma_b = 1., 1.

for i in range(len(sols)):
    s3.append(sof_prob_dom_multi2(sols, sols[i], sigma_a, sigma_b))
s3

[2.2906164496671124,
 1.634060344553688,
 1.436565639361921,
 1.9073932584275828,
 3.1988455322854907,
 4.532518775704205]

In [77]:
s3 = []
sigma_a, sigma_b = 0.05, 0.05


for i in range(len(y_samples)):
    s3.append(sof_prob_dom_multi2(y_samples, y_samples[i], sigma_a, sigma_b))
s3

[9.983539862158816,
 14.923115396531195,
 15.100614790037495,
 13.991592720363448,
 17.605723494257724,
 16.413209176635547,
 16.902782142840184,
 16.73786017316808,
 12.26186565188478,
 15.017493542825521,
 13.958191127710052,
 14.264908771016616,
 17.750210555379986,
 10.70573645874667,
 11.040314429828666,
 11.64130724073004,
 16.07685479955206,
 13.598026547638694,
 18.27880863909876,
 13.747844479595656]

In [78]:
np.sum(s3)

290.0

In [79]:
sum_of_rank_pos(y_samples.shape[0])

190.0

### Everything kinda works?

* expect the m needs to be * by 10 for the paper examples to work

* and one should minus -5.5 from the sum with 5 var, 3 obj to keep sum of rank pos equal.


Probably has something to do with below..
"""
domination calculations.
If we consider the ranking case for a single dimension, if A and B are iden-
tical, we cannot choose between them and so add in 0.5 to the sum. With
non-domination, we also have the situation where we cannot choose between
objectives and should therefore add 0.5 to the sum as required. In the case of
uncertain measurements, we can multiply the value of 0:5 by the probability of
non-domination, and still subtract o 0:5 to allow for comparing the individual
with itself, thereby maintaining the sum of the rank positions as n(n 1)=2.
Thus we can add the non-domination term into (21). The rank calculation for
multi-objective ranking is shown in (22), where n is the number of measurements

""
Does it just come from that 5.5 happens to be the combinations of 3 dimensions in this case?

In [ ]:
x = 0.5 * 2
0.25

0.25 = y * 5

0.25/3

In [80]:
z = 5.5 * 5

In [81]:
z

27.5

In [82]:
0.5*3

1.5

In [83]:
0.5*1.5

0.75

In [84]:
5.5 / 1.5

3.6666666666666665

In [85]:
5.5 / 3


1.8333333333333333